In [1]:
import pandas as pd
import chardet
import re
import bs4
import requests
import csv

In [2]:
with open('hasil_scraping.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

In [3]:
films = pd.read_csv('hasil_scraping.csv', encoding = 'ISO-8859-1')
films

,Judul,Tahun,Durasi,Rating IMDB,Metascore,Genre,Gross($M)
0,The Shawshank Redemption,1994,142,9.3,80,Drama,28.34
1,The Dark Knight,2008,152,9.0,84,Action,534.86
2,Inception,2010,148,8.8,74,Action,292.58
3,Fight Club,1999,139,8.8,66,Drama,37.03
4,Pulp Fiction,1994,154,8.9,94,Crime,107.93
...,...,...,...,...,...,...,...
245,Rain Man,1988,133,8.0,65,Drama,178.80
246,Ex Machina,2014,108,7.7,78,Drama,25.44
247,(500) Days of Summer,2009,95,7.7,76,Comedy,32.39
248,"Crazy, Stupid, Love.",2011,118,7.4,68,Comedy,84.35


In [4]:
imdb = pd.read_csv('imdb.csv')
imdb

,Name,Date,Rate,Votes,Genre,Duration,Type,Certificate,Episodes,Nudity,Violence,Profanity,Alcohol,Frightening
0,No Time to Die,2021,7.6,"107,163","Action, Adventure, Thriller",163,Film,PG-13,-,Mild,Moderate,Mild,Mild,Moderate
1,The Guilty,2021,6.3,"64,375","Crime, Drama, Thriller",90,Film,R,-,None,None,Severe,None,Moderate
2,The Many Saints of Newark,2021,6.4,"27,145","Crime, Drama",120,Film,R,-,Moderate,Severe,Severe,Moderate,Moderate
3,Venom: Let There Be Carnage,2021,6.4,"30,443","Action, Adventure, Sci-Fi",97,Film,PG-13,-,None,Moderate,Moderate,Mild,Moderate
4,Dune,2021,8.3,"84,636","Action, Adventure, Drama",155,Film,PG-13,-,None,Moderate,None,Mild,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6173,The Human Centipede II (Full Sequence),2011,3.8,"37,492",Horror,91,Film,Not Rated,-,Severe,Severe,Severe,Mild,Severe
6174,Double Indemnity,1944,8.3,"150,448","Crime, Drama, Film-Noir",107,Film,Passed,-,None,Mild,None,Mild,Mild
6175,Before the Devil Knows You're Dead,2007,7.3,"100,668","Crime, Drama, Thriller",117,Film,R,-,Severe,Moderate,Severe,Severe,Severe
6176,Queen Bees,2021,6.0,887,"Comedy, Drama, Romance",100,Film,PG-13,-,None,None,Mild,Moderate,None


In [5]:
titles = films['Judul']
year = films['Tahun']
age = imdb['Certificate']
rate = imdb['Rate']
print(titles)
print(year)
print(age)
print(rate)

0                    The Shawshank Redemption
1                             The Dark Knight
2                                   Inception
3                                  Fight Club
4                                Pulp Fiction
                        ...                  
245                                  Rain Man
246                                Ex Machina
247                      (500) Days of Summer
248                      Crazy, Stupid, Love.
249    Harry Potter and the Half-Blood Prince
Name: Judul, Length: 250, dtype: object
0      1994
1      2008
2      2010
3      1999
4      1994
       ... 
245    1988
246    2014
247    2009
248    2011
249    2009
Name: Tahun, Length: 250, dtype: object
0           PG-13
1               R
2               R
3           PG-13
4           PG-13
          ...    
6173    Not Rated
6174       Passed
6175            R
6176        PG-13
6177            R
Name: Certificate, Length: 6178, dtype: object
0       7.6
1       6.3
2       6.4
3

In [6]:
links = []

for n in range(0,250):
    rd = titles[n]
    rd = rd.encode('unicode_escape')
    rd = rd.decode('ISO-8859-1')
    #if (chardet.detect(rd) != 'utf-8'):
            #print('ok')
    #titles[n] = rd
    print(rd)
    x = re.sub(' ', '+', rd)
    search = 'https://www.imdb.com/find?q=' + x + '&ref_=nv_sr_sm'
    try:
        response = requests.get(search)
        soup = bs4.BeautifulSoup(response.text, 'lxml')
        links.append(soup.select('td.result_text a')[0].attrs.get('href'))
    except:
        break

In [7]:
len(links.unique())

250

In [8]:
titleList = []
ageList = []
yrList = []
for n in range(0,250):
    stop = 1
    for z in range(0, len(age)):
        if (titles[n] == imdb['Name'][z]) & (imdb['Type'][z] == 'Film'):
            '''if (year[n] != str(imdb['Date'][z])):
                print(titles[n], year[n], imdb['Date'][z])
                print(titles[n], year[n], imdb['Date'][z])
            else:'''
            titleList.append(titles[n])
            ageList.append(age[z])
            yrList.append(imdb['Date'][z])
            stop = 0
    if stop == 1:
        titleList.append(titles[n])
        ageList.append(None)
        yrList.append(year[n])

dict = {'Title': titleList, 'AgeRating': ageList, 'Year': yrList}

cert = pd.DataFrame(data = dict)
cert

,Title,AgeRating,Year
0,The Shawshank Redemption,R,1994
1,The Dark Knight,PG-13,2008
2,Inception,PG-13,2010
3,Fight Club,R,1999
4,Pulp Fiction,R,1994
...,...,...,...
248,Rain Man,R,1988
249,Ex Machina,R,2014
250,(500) Days of Summer,PG-13,2009
251,"Crazy, Stupid, Love.",PG-13,2011


In [9]:
rep = []
for n in range(0, len(cert)):
    if cert["Title"].duplicated()[n]:
        if (year[n] != cert['Year'][n]):
            cert = cert.drop(index = n)
cert

,Title,AgeRating,Year
0,The Shawshank Redemption,R,1994
1,The Dark Knight,PG-13,2008
2,Inception,PG-13,2010
3,Fight Club,R,1999
4,Pulp Fiction,R,1994
...,...,...,...
248,Rain Man,R,1988
249,Ex Machina,R,2014
250,(500) Days of Summer,PG-13,2009
251,"Crazy, Stupid, Love.",PG-13,2011


In [10]:
'''f = open('Cert.csv', 'w')
writer = csv.writer(f)
head = ['Title', 'AgeRating', 'Year']
writer.writerow(head)
for i in range(0, len(cert)):
    writer.writerow(cert.iloc[i])
f.close()'''

"f = open('Cert.csv', 'w')\nwriter = csv.writer(f)\nhead = ['Title', 'AgeRating', 'Year']\nwriter.writerow(head)\nfor i in range(0, len(cert)):\n    writer.writerow(cert.iloc[i])\nf.close()"

In [11]:
f = open('Films.csv', 'w')
writer = csv.writer(f)

head = ['Title', 'AgeRating', 'Languages']
writer.writerow(head)

for i in range(0, len(links)):
    lang = []
    last = []
    web = 'https://www.imdb.com' + links[i]
    response = requests.get(web)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    
    title = soup.find_all('h1')
    n = soup.find_all(href=re.compile("language"))
    
    #for t in title:
        #last.append(t.contents[0])
        #last.append(t.contents)
    for m in n:
        for j in range(0, len(m)):
            lang.append(m.contents[j])
    last.append(cert.iloc[i]['Title'])
    last.append(cert.iloc[i]['AgeRating'])
    last.append(lang)
    writer.writerow(last)
    #lang.append(last)
    
f.close()